In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv(r'C:\Users\Lenovo\Downloads\Bitext_Sample_Customer_Service_Training_Dataset.xlsx', encoding='ISO-8859-1')


ParserError: Error tokenizing data. C error: Expected 2 fields in line 12, saw 5


In [5]:
import pandas as pd

df = pd.read_excel(r'C:\Users\Lenovo\Downloads\Bitext_Sample_Customer_Service_Training_Dataset.xlsx')


In [6]:
len(df.intent.value_counts())

27

In [7]:
df = df.drop(columns=['flags', 'category'])

In [8]:
df

,utterance,intent
0,I have problems with canceling an order,cancel_order
1,how can I find information about canceling ord...,cancel_order
2,I need help with canceling the last order,cancel_order
3,could you help me cancelling the last order I ...,cancel_order
4,problem with cancelling an order I made,cancel_order
...,...,...
8170,I need help to check if there is any updates o...,track_refund
8171,refund status,track_refund
8172,how can I check the status of the refund?,track_refund
8173,help checking if there is anything wrong with ...,track_refund


In [9]:
df.utterance[0]

'I have problems with canceling an order'

In [10]:
import nltk
import matplotlib.pyplot as plt
import random
import re
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [11]:
def preprocess_utterance(utterance):
    stemmer = PorterStemmer()
    stopwords_eng = stopwords.words('english') 
    
    utterance_tokens = word_tokenize(utterance)
    utterance_tokens_clean = []
    for token in utterance_tokens:
        if (token not in stopwords_eng and token not in string.punctuation):
            stem_token = stemmer.stem(token)
            utterance_tokens_clean.append(stem_token.lower())
            
    return utterance_tokens_clean

In [12]:
df.utterance = df.utterance.apply(preprocess_utterance)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\Lenovo/nltk_data'
    - 'C:\\Program Files\\Python312\\nltk_data'
    - 'C:\\Program Files\\Python312\\share\\nltk_data'
    - 'C:\\Program Files\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\Lenovo\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [14]:
df

,utterance,intent
0,I have problems with canceling an order,cancel_order
1,how can I find information about canceling ord...,cancel_order
2,I need help with canceling the last order,cancel_order
3,could you help me cancelling the last order I ...,cancel_order
4,problem with cancelling an order I made,cancel_order
...,...,...
8170,I need help to check if there is any updates o...,track_refund
8171,refund status,track_refund
8172,how can I check the status of the refund?,track_refund
8173,help checking if there is anything wrong with ...,track_refund


In [15]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, stratify=df.intent)

In [16]:
df_train = df_train.sort_values('intent').reset_index(drop=True)

In [17]:
df_test = df_test.sort_values('intent').reset_index(drop=True)

In [18]:
df_test

,utterance,intent
0,can you help me canceling the last order?,cancel_order
1,where can I cancel the last order I made?,cancel_order
2,I have problems with canceling the last order ...,cancel_order
3,I want to cancel the order I made,cancel_order
4,how can I cancel an order I made?,cancel_order
...,...,...
1630,I try to track the refund,track_refund
1631,I don't know what I need to do to track the re...,track_refund
1632,I would like to track the refund,track_refund
1633,assistance checking the status of the refund,track_refund


In [19]:
intents = list(df_train.intent.unique())

In [20]:
df_train = df_train.replace(intents, range(0, 27))

In [21]:
df_test = df_test.replace(intents, range(0, 27))

In [22]:
freqs = {}
for utterance, intent in list(df_train.itertuples(index=False)):
    for word in utterance:
        pair = (word, intent)
        if pair in freqs:
            freqs[pair] += 1
        else:
            freqs[pair] = 1

TypeError: 'int' object is not iterable

In [23]:
keys = []
for freq in freqs:
    if freq[0] not in keys:
        keys.append(freq[0])

In [24]:
freq_data = []

for key in keys:
    intent_freqs = [0]*27
    for i in range(0, 27):
        if (key, i) in freqs:
            intent_freqs[i] = freqs[(key, i)]
    freq_data.append([key] + intent_freqs)